### team members: 
- Syed Razauddin Shahlal ;         ubid: 50496396
- Tajammul Shuja Sayyad  ;         ubid: 50495179  

### Transformers based model 

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fakenews/WELFake_Dataset.csv
/kaggle/input/fake-news-classification/WELFake_Dataset.csv


##### install the required libraries. We will be using hugging face transformers library.

In [2]:
!pip install transformers evaluate datasets

  Obtaining dependency information for evaluate from https://files.pythonhosted.org/packages/70/63/7644a1eb7b0297e585a6adec98ed9e575309bb973c33b394dae66bc35c69/evaluate-0.4.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [3]:
#!pip install evaluate


In [4]:
import pandas as pd
df=pd.read_csv("/kaggle/input/fakenews/WELFake_Dataset.csv")


In [5]:
# drop nan values from both text and title as there are very few missing values so we will remove those rows
df=df.dropna()

##### check the distribution of the target variable.

In [6]:
#check label distribution--> equal distribution for both classes.
df['label'].value_counts()

label
1    36509
0    35028
Name: count, dtype: int64

##### Splitting our data(70-15-15)
- training data: used for fine tuning the models
- validation data: for hyperparameters tuning
- testing data: for checking accuracy and measuring generalizability.

In [7]:
#we will divide our dataset into train validation and test sets. 
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.3,random_state=42)
test_df, eval_df = train_test_split(test_df, test_size=0.5,random_state=42)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### for hugging face transformers we will need Dataset object


In [8]:
# for hugging face transformers we will need Dataset object
from datasets import Dataset


In [9]:
#convert our data from dataframe to datasets(for hugging face library implementations)
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
eval_dataset = Dataset.from_pandas(eval_df)

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [10]:
train_dataset

Dataset({
    features: ['Unnamed: 0', 'title', 'text', 'label', '__index_level_0__'],
    num_rows: 50075
})

In [11]:
# we will remove unwanted columns which we dont require for prediction.
train_dataset=train_dataset.remove_columns(['Unnamed: 0','__index_level_0__'])
test_dataset=test_dataset.remove_columns(['Unnamed: 0','__index_level_0__'])
eval_dataset=eval_dataset.remove_columns(['Unnamed: 0','__index_level_0__'])

##### We will first try just with the title column and experiment with different models to get a sense how each model is doing.

In [12]:
#first try with just title to predict(epoch=2)-->distilebert,bert

In [13]:
# each pretrained transformer model comes with its pretrained tokenizer.
# we will use these tokenizers to tokenize our data so that it aligns with model's input requirements.
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

##### define a preprocess function

In [14]:
# define a preprocess function that takes the dataset as parameter and then we will tokenize 'title' col.
# truncation is set to true so that if a text has token length more than models token length limit it will be truncated(512 for bert)
def preprocess_function(examples):
    return tokenizer(examples["title"], truncation=True)

##### tokenize each of the datasets.
- To process dataset in one step, use hugging face Datasets map method to apply a preprocessing function over the entire dataset


In [15]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [16]:
# we can see that distil-bert tokenized the data and added input_ids and attention_mask as features.
tokenized_train_dataset

Dataset({
    features: ['title', 'text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 50075
})

#####  looking at input_ids features:
- 101 :[cls] token(added at beginning of each sentence) and 
- 102 :[sep] token(Added at the end) added by distel-bert tokenizer

In [17]:
#we can look at an example. 101 is the cls token and 102 sep token added by distel-bert.
tokenized_train_dataset[0]['input_ids']

[101,
 8499,
 787,
 188,
 5055,
 1113,
 15978,
 13429,
 24442,
 7426,
 4785,
 6424,
 6335,
 2409,
 7661,
 787,
 188,
 118,
 1109,
 1203,
 1365,
 2706,
 102]

##### Data collator for padding:
- It’s more efficient to dynamically pad the sentences to the longest length in a batch during collation, instead of padding the whole dataset to the maximum length.

In [18]:
# data collator is required for data of varied lengths.
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
# we will use accuracy as the evaluation metric
import evaluate

accuracy = evaluate.load("accuracy")

##### define computing metrics function

In [20]:
# models output are raw logits so we will have to convert them into predicted labels by using argmax function.
# essentially we have an output that we get after model applies softmax function to the last layer/output layer(Also called head)
# these logits values are probabilty distribution.
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [21]:
#create a dictionary to represent class labels.
id2label = {0: "FAKE", 1: "REAL"}
label2id = {"FAKE": 0, "REAL": 1}

##### We will first try Distilbert model which is a lighter version of bert.

In [22]:
#load the distilbert model
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# we can check the models architecture
print(model.config)

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "FAKE",
    "1": "REAL"
  },
  "initializer_range": 0.02,
  "label2id": {
    "FAKE": 0,
    "REAL": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.35.0",
  "vocab_size": 30522
}



In [24]:
# as first we are trying just by title column we can drop text column. Although the Trainer class can take of this itself.
tokenized_train_dataset=tokenized_train_dataset.remove_columns(['text'])
tokenized_eval_dataset=tokenized_eval_dataset.remove_columns(['text'])
tokenized_test_dataset=tokenized_test_dataset.remove_columns(['text'])

##### Instantiate Training arguments and Trainer
- Training Arguments: create a TrainingArguments class which contains all the hyperparameters you can tune as well as flags for activating different training options.

- Trainer class: Hugging face Transformers provides a Trainer class optimized for training Transformers models, making it easier to start training without manually writing your own training loop. The Trainer API supports a wide range of training options and features such as logging, gradient accumulation, and mixed precision.

In [25]:
# we start with 2 epochs to build our model

In [26]:
training_args = TrainingArguments(
    output_dir="model_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.168900,0.155389,0.937937
2,0.112300,0.145969,0.944274


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=3130, training_loss=0.16893480501997585, metrics={'train_runtime': 546.6381, 'train_samples_per_second': 183.211, 'train_steps_per_second': 5.726, 'total_flos': 1377687077286588.0, 'train_loss': 0.16893480501997585, 'epoch': 2.0})

In [27]:
#next we check accuracy on test data by looking at eval_accuracy.
trainer.evaluate(tokenized_test_dataset)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.1364707350730896,
 'eval_accuracy': 0.9485602460162147,
 'eval_runtime': 16.4035,
 'eval_samples_per_second': 654.191,
 'eval_steps_per_second': 20.483,
 'epoch': 2.0}

##### Observations:
- just by using title column and fine tuning distilbert we are getting accuracy of around 94% for validation and around 95% for test data

### Now we try bert with title column
- for bert we use its own tokenizer which it was pre trained on to get the embeddings 

In [28]:
#with bert with just title column
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

tokenized_train_dataset=tokenized_train_dataset.remove_columns(['text'])
tokenized_eval_dataset=tokenized_eval_dataset.remove_columns(['text'])
tokenized_test_dataset=tokenized_test_dataset.remove_columns(['text'])

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
training_args = TrainingArguments(
    output_dir="model_1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.168700,0.158426,0.938589
2,0.109600,0.150940,0.946324


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=3130, training_loss=0.16876022960431278, metrics={'train_runtime': 929.4071, 'train_samples_per_second': 107.757, 'train_steps_per_second': 3.368, 'total_flos': 2736406878510780.0, 'train_loss': 0.16876022960431278, 'epoch': 2.0})

In [30]:
trainer.evaluate(tokenized_test_dataset)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.14027650654315948,
 'eval_accuracy': 0.950796757058988,
 'eval_runtime': 28.8723,
 'eval_samples_per_second': 371.671,
 'eval_steps_per_second': 11.637,
 'epoch': 2.0}

##### observations:
- we observe slight better accuracy for both test and validation with bert.

### Using both text and title columns

In [31]:
# we get slight improvement just by using the title column. Next we will combine both title and text.


##### distilbert with both title and text

In [32]:
def preprocess_function(examples):
    return tokenizer(examples["title"],examples["text"], truncation=True)

In [33]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [34]:
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

##### When we comine text and title berts tokenizer adds speacial token like cls at the beginning and sep token at the end of one text data.
- To differentiate between title and text bert added [sep] token after title ended.

In [37]:
#lets have a look how does the features look like after tokenizing both text and title.
#bert's tokenizer adds special tokens to differentiate different texts. 
#Here we can see title and text are seperated by SEP token.
tokenizer.decode(tokenized_train_dataset[0]['input_ids'])

'[CLS] Trump ’ s Take on Corporate Tax Rate Could Look Very Much Like Obama ’ s - The New York Times [SEP] Just a few days after being sworn in as president, Donald J. Trump convened a meeting at the White House of some of the nation ’ s most prominent chief executives to discuss how to improve manufacturing. Mr. Trump was joined by Elon Musk of Tesla, Mark Fields of Ford, Andrew Liveris of Dow Chemical, Marillyn A. Hewson of Lockheed Martin and Michael Dell of Dell, among others. Before the meeting formally began, with cameras rolling, Mr. Trump wanted to talk about corporate tax rates — specifically lowering them. During one debate with Hillary Clinton, he had told voters : “ Under my plan, I ’ ll be reducing taxes tremendously, from 35 percent to 15 percent for companies, small and big businesses. That ’ s going to be a job creator like we haven ’ t seen since Ronald Reagan. It ’ s going to be a beautiful thing to watch. ” He repeated the 15 percent figure over and over again. Howev

In [38]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
training_args = TrainingArguments(
    output_dir="model_2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.072400,0.065378,0.979965


TrainOutput(global_step=1565, training_loss=0.1604620979235957, metrics={'train_runtime': 1483.3769, 'train_samples_per_second': 33.757, 'train_steps_per_second': 1.055, 'total_flos': 6633304987699200.0, 'train_loss': 0.1604620979235957, 'epoch': 1.0})

In [40]:
#on test data
trainer.evaluate(tokenized_test_dataset)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.0713673010468483,
 'eval_accuracy': 0.9778212654924984,
 'eval_runtime': 111.1106,
 'eval_samples_per_second': 96.579,
 'eval_steps_per_second': 3.024,
 'epoch': 1.0}

##### observations:
- We get 97% accuracy after 1 epoch when we combine text and title columns.

##### finally we use bert on both the features(title+text)

In [41]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_test_dataset = test_dataset.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)



  0%|          | 0/51 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [42]:
#with bert tokenizer we get an additional feature known as token_type_ids
tokenized_train_dataset

Dataset({
    features: ['title', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 50075
})

##### understanding bert's tokenizers features:
- input_ids: these are numbers that are given to each word in our data according to the bert's vocabulary.
- attention masks: tells bert that if a text was padded(to have a fixed length input) it should not be focused on. It uses binary list of 1's and 0's. 0 tells us that the word was padded.
- token_type_ids: model can differentiate between title input_ids and text input_ids.

In [44]:
#this is a list with 0 and 1 which tells model which input_ids belong to which column. 
#0 here is for title and 1 for text column input_ids.
#this way model can know if there are multiple text inputs and can differentiate them.
tokenized_train_dataset[0]['token_type_ids']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [45]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "bert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [46]:
training_args = TrainingArguments(
    output_dir="model_3",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,0.062900,0.063986,0.982108


TrainOutput(global_step=1565, training_loss=0.1292401312258297, metrics={'train_runtime': 2948.1508, 'train_samples_per_second': 16.985, 'train_steps_per_second': 0.531, 'total_flos': 1.3175286097152e+16, 'train_loss': 0.1292401312258297, 'epoch': 1.0})

In [47]:
#next we check accuracy on test data by looking at eval_accuracy.

trainer.evaluate(tokenized_test_dataset)


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.06537528336048126,
 'eval_accuracy': 0.9819215357375827,
 'eval_runtime': 218.3003,
 'eval_samples_per_second': 49.157,
 'eval_steps_per_second': 1.539,
 'epoch': 1.0}

##### Observations:
- Just by training for 1 epoch we can see bert is doing a great job and we are getting 98% accuracy for both validation and test data
- Bert with both the features is doing the best job.

##### FInal thoughts:
- Depending on the resources and time available we can train the model for even more epochs and apply early stopping if we see that the model's performance is not changing over time.
- We can also look at models that are more sophisticated than bert like roberta, XL-Net and transformer-XL

##### references:
- https://huggingface.co/docs/transformers/index
